In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
import transformers
import os
import pandas as pd
import pinecone
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# GT
qa = pd.read_csv('./data/gene_qa.csv', encoding='ISO-8859-1')
qa = qa[['question','answer']]

# external data
loader = TextLoader("./data/gene_revised.csv")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents) # all_splitters

# pinecone setting & upsert
PINECONE_API_KEY = '6e7208af-6071-4493-854f-4ea43a3c5d53' # snu 계정 pinecone api_key
PINECONE_ENV = 'gcp-starter'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# make pinecone index
index_name = 'gene-index'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

# embedding
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# upload vector db
vectordb = Pinecone.from_documents(docs, embeddings, index_name=index_name)
# if already have db 
# Pinecone.from_existing_index(index_name, embeddings)

# define llm
###### model load 필요 ########

###########
generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
) # config

llm = HuggingFacePipeline(pipeline=generate_text)

chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True)

cuda


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
import pandas as pd
gene = pd.read_csv('./data/gene_revised.csv')
data = gene['description'].tolist()
data[0]

'The AAAS gene provides instructions for making a protein called ALADIN whose function is not well understood. Within cells, ALADIN is found in the nuclear envelope, the structure that surrounds the nucleus and separates it from the rest of the cell. Based on its location, ALADIN is thought to be involved in the movement of molecules into and out of the nucleus.'

In [ ]:
# gene_data load code
# import json
# from langchain.document_loaders import WebBaseLoader

# with open('gene_url.json', 'r') as file:
#    data = json.load(file)
# web_links = data
# loader = WebBaseLoader(web_links)
# documents = loader.load()

# #to save loaded documents to csv file
# import csv

# with open('gene.csv', 'w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerow(['col'])
#    for item in documents:
#        writer.writerow([item])
# gene = pd.read_csv('./gene.csv')
# data = gene['col'].tolist()

In [68]:
# gene data preprocessing code

# gene = pd.read_csv('./gene.csv')
# data = gene['col'].tolist()

# import re

# revises = []

# for i, d in enumerate(data):
#     try:
#         start_idx = re.search(r'\\nThe',d).start()+2
#         revise = d[start_idx:]
#         end_idx =re.search(r'\\n\\n\\n\\nHealth',revise).start()
#         revise = revise[:end_idx]
#         revises.append([revise])
#     except:
#         start_idx = re.search(r'description',e).end() + 3
#         r_e = e[start_idx:]
#         end_idx = re.search(r'Learn',r_e).start()
#         revise = r_e[:end_idx]
#         revises.append([revise])


# print(revises)

# # save to csv

# import csv

# fields = ['description']

# with open('gene_revised.csv', 'w', newline='') as csv_file:
#     writer = csv.writer(csv_file)
#     writer.writerow(fields)
#     writer.writerows(revises)

In [3]:
import pandas as pd
qa = pd.read_csv('./gene_qa.csv', encoding='ISO-8859-1')
qa = qa[['question','answer']]
qa

,question,answer
0,"Acrodermatitis enteropathica, Birk-Landau-Pere...",Zinc
1,Which of the following would represent a frame...,c.169_170insA
2,"On examination, a patient is noted to have ¡°c...",Urine glycosaminoglycans
3,"A man and a woman have 2 children, both of who...",12.50%
4,An infant was slightly premature and is being ...,Williams syndrome
...,...,...
79,The wings of a bat and a bird have similar for...,Analogous
80,You are analyzing whole genome sequencing data...,Cornelia de Lange syndrome
81,A teen-ager has a several year history of inte...,Dentatorubral-pallidoluysian atrophy (DRPLA)
82,In general (and with the caveat that there can...,Dravet syndrome


In [8]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs)

In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("What is ABCA12 gene? if you refer document, please answer with reference part")
print(response)

The ABCA12 gene provides instructions for making a protein known as an ATP-binding cassette (ABC) transporter. This information is mentioned in the document.


In [ ]:
index = SummaryIndex.from_documents(docs)

query_engine = index.as_query_engine()
response = query_engine.query("What is ABCA12 gene? ")
display(Markdown(f"<b>{response}</b>"))